# NLP project


Project 21: Automatic Summarization  

We shall consider structured document containing a title, abstract and a set of subsections. We would like to build a text summarizer such that tracks important keywords in the document. For this purpose, the first step is identify these keywords.  

In [1]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip list
#tarkista löytyykö: lxml, html5lib, requests

In [ ]:
#jos nltk ei löydy asenna -> ! pip install nltk
import nltk
#nltk.download("stopwords")
#from nltk.cluster.util import cosine_distance

## TASK 1
Assume the initial input is given as html document (choose an example of your own), we hypothesize that important keywords are initially contained in the words of titles, abstract and possibly titles of subsections of the document. Suggest a simple python script that inputs an html document and outputs the lists of words in the title, abstract and title of section/subsections.

In [6]:
from bs4 import BeautifulSoup
import requests


url = "https://iltalehti.fi" 
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')

print(soup.prettify)

#jos saat errorin -> FeatureNotFound, pakko päivitä lxml käskyllä -> pip install lmxl --upgrade 




<bound method Tag.prettify of <!DOCTYPE html>
<html class="no-js" lang="fi"><head><meta charset="utf-8"/><meta content="ie=edge" http-equiv="x-ua-compatible"/><link as="font" crossorigin="anonymous" href="https://assets.ilcdn.fi/BerninoSans-Web-CondensedBold.woff2" rel="preload" type="font/woff2"/><link as="font" crossorigin="anonymous" href="https://assets.ilcdn.fi/BerninoSans-Web-Bold.woff2" rel="preload" type="font/woff2"/><link as="font" crossorigin="anonymous" href="https://assets.ilcdn.fi/BerninoSans-Web-Semibold.woff2" rel="preload" type="font/woff2"/><link as="font" crossorigin="anonymous" href="https://assets.ilcdn.fi/BerninoSans-Web-CondensedSemibold.woff2" rel="preload" type="font/woff2"/><link href="https://static.ilcdn.fi" rel="dns-prefetch"/><link href="https://img.ilcdn.fi" rel="dns-prefetch"/><link href="https://api.il.fi" rel="dns-prefetch"/><title>Iltalehti | IL - Suomen suurin uutispalvelu</title><meta content="Uutiset, urheilu, viihde, talous, sää, terveys, ruoka, m